最小化 Host -> <- device 数据传输

device 共享内存等价于用户管理的缓存

1.将数据从device内存载入共享内存
2.Block块内的每个线程，
2.1数据从全局内存加载到共享内存
2.2同步Block内的所有其他线程，（以便安全读取共享内存数据，可被其他线程写入）
2.3在共享内存中处理数据
2.4必要时再次同步，确保结果被update
2.5将结果写入到Device 的全局内存中

计算能力8.9 4090 最多配置99KB共享内存。






方式2

Host移动更多codes 到Device，
尽管这样会使kernel函数在device上的并行程度降低，使用率降低。在Device的内存上可能会产生中间的数据结构，device会操作这个数据结构，
这个数据结构可能被销毁，不被映射到主机、复制到host内存。



方式3
将多次小传输打包到单一大传输。



方式4 
使用PageHostMemory ，提高性能。

当使用MappedMemory时，没必要分配Device内存，也无需显式复制数据。

方式5 统一内存
略，目前仅苹果。

Device

GlobalMemory

对齐要求

Shared Memory
带宽更大，
当访问bank冲突时，将并行访问变为顺序访问。降低吞吐量

Constant Memory
常量内存映射到Device上，缓存在常量缓存区内。


Texture & surface memory
对2dim定位有优化。


高效指令Throughput

最小化低吞吐量的算数指令。使用指令集而非常规函数，使用单精度而非双精度。

__log2f
exp2f 
__sinf
__cosf

__[u]mul24


SIME video instructions 
vabsdiff2
vabsdiff4

math_functions.h
device_functions.h
通常来说的话，编译code时，比起编译选项-ftz=false，开启编译选项-ftz=true会有更高的性能（denormalized numbers are flushed to zero）。类似地，比起编译选项-prec-div=true，使用编译选项-prec-div=false（less precise division低精度的除）会有更高的性能。比起编译选项-prec-sqrt=true，使用编译选项-prec-sqrt=false(less precise square root低精度开方)会有更高的性能。nvcc用户手册更加详细的描述了这些编译选项。

Single-Precision Floating-Point Division

__fdividef(x,y)（见Intrinsic Functions）提供了更加快速的单精度浮点数除，比起除算子。


Single-Precision Floating-Point Reciprocal Square Root

要遵守标准IEEE-754的内容，编译器可以优化1.0/sqrtf()到rsqrtf()，当且仅当reciprocal 和 square root都是近似计算的。（像：-prec-div=false 和 -prec-sqrt=false），所以当满足条件是最好直接调用函数rsqrtf()。

Single-Precision Floating-Point Square Root

单精度浮点数开方，实现方式是Reciprocal Square Root，然后又跟了一个Reciprocal，而不是Reciprocal Square Root后面跟了一个乘积，所以能给出0或者infinity的准确结果。

Sine and Cosine

sinf(x), cosf(x), tanf(x)和sincosf(x)函数，和相应的双精度浮点数指令，消耗更大些，甚至消耗会更大当输入的参数数量级很大时。

更准确的来说，参数规约的code（实现见Mathematical Functions），组合了两个code path，一个是fast path，另一个则是slow path。

最小化控制流程指令  if switch 

减少指令数量
restricted指针 synchronization Instruction优化同步点。